# Chapter 53: WebAssembly Integration

WebAssembly (WASM) brings near-native performance to web applications, enabling compute-intensive tasks like image processing, cryptography, and data compression to run at speeds impossible with JavaScript alone. For Next.js applications, WASM offers a pathway to optimize performance-critical paths while maintaining the developer experience of React. This chapter explores integrating Rust-compiled WASM modules, managing the JavaScript/WASM boundary, and deploying performance modules in production environments.

By the end of this chapter, you'll understand the WebAssembly binary format and linear memory model, compile Rust code to WASM using wasm-bindgen, dynamically load WASM modules in Next.js to avoid SSR conflicts, implement high-performance image processing and hashing algorithms, manage memory transfer between JavaScript and WASM efficiently, debug WASM modules using browser DevTools, and evaluate the evolving WASI standard for server-side WASM.

## 53.1 WebAssembly Fundamentals

Understanding the low-level architecture of WASM modules.

### The WebAssembly Binary Format

How WASM works under the hood:

```typescript
// WebAssembly is a binary instruction format for a stack-based virtual machine.
// Key characteristics:
// - Binary format (smaller than text-based JS)
// - Near-native performance (no interpretation overhead)
// - Memory-safe sandboxed execution environment
// - Language agnostic (compile from Rust, C++, Go, etc.)

// WASM Module Structure:
interface WasmModule {
  // Linear memory - contiguous array of bytes
  memory: WebAssembly.Memory;
  
  // Exported functions callable from JavaScript
  exports: {
    add: (a: number, b: number) => number;
    process_image: (ptr: number, len: number) => number;
  };
  
  // Imports from host (JavaScript)
  imports: {
    env: {
      abort: () => void;
      console_log: (ptr: number) => void;
    };
  };
}

// Loading a WASM module in JavaScript
async function loadWasm() {
  // Fetch the binary
  const response = await fetch('/wasm/calculator.wasm');
  const buffer = await response.arrayBuffer();
  
  // Instantiate with imports (host functions)
  const module = await WebAssembly.instantiate(buffer, {
    env: {
      memory: new WebAssembly.Memory({ initial: 256 }), // 16MB pages
      abort: () => console.error('WASM aborted'),
    },
  });
  
  return module.instance.exports;
}
```

### Linear Memory Management

The shared memory buffer between JavaScript and WASM:

```rust
// Rust code (src/lib.rs)
use wasm_bindgen::prelude::*;

// Export function to JavaScript
#[wasm_bindgen]
pub fn process_data(data: &[u8]) -> Vec<u8> {
    // data is a view into WASM linear memory
    let mut result = Vec::new();
    for byte in data {
        result.push(byte.wrapping_add(1)); // Simple transform
    }
    result
}

// Manual memory management example
#[wasm_bindgen]
pub struct ImageProcessor {
    width: u32,
    height: u32,
    buffer: Vec<u8>,
}

#[wasm_bindgen]
impl ImageProcessor {
    #[wasm_bindgen(constructor)]
    pub fn new(width: u32, height: u32) -> ImageProcessor {
        let size = (width * height * 4) as usize;
        ImageProcessor {
            width,
            height,
            buffer: vec![0; size],
        }
    }
    
    // Returns pointer to buffer (for JS to write image data)
    #[wasm_bindgen(js_name = getBufferPtr)]
    pub fn get_buffer_ptr(&mut self) -> *mut u8 {
        self.buffer.as_mut_ptr()
    }
    
    pub fn process(&mut self) {
        // Modify buffer in place
        for i in (0..self.buffer.len()).step_by(4) {
            // Simple grayscale
            let gray = (self.buffer[i] as u16 + 
                       self.buffer[i + 1] as u16 + 
                       self.buffer[i + 2] as u16) / 3;
            self.buffer[i] = gray as u8;
            self.buffer[i + 1] = gray as u8;
            self.buffer[i + 2] = gray as u8;
            // Alpha remains
        }
    }
}
```

```typescript
// JavaScript interaction with linear memory
// lib/wasm/memory.ts

export class WasmMemoryManager {
  private memory: WebAssembly.Memory;
  private exports: any;
  
  constructor(memory: WebAssembly.Memory, exports: any) {
    this.memory = memory;
    this.exports = exports;
  }
  
  // Write JavaScript data to WASM memory
  writeString(str: string): number {
    const encoder = new TextEncoder();
    const bytes = encoder.encode(str);
    const ptr = this.exports.alloc(bytes.length + 1); // Rust allocator
    
    const view = new Uint8Array(this.memory.buffer, ptr, bytes.length + 1);
    view.set(bytes);
    view[bytes.length] = 0; // Null terminator
    
    return ptr;
  }
  
  // Read string from WASM memory
  readString(ptr: number): string {
    const view = new Uint8Array(this.memory.buffer);
    let end = ptr;
    while (view[end] !== 0) end++;
    
    const decoder = new TextDecoder();
    return decoder.decode(view.subarray(ptr, end));
  }
  
  // Write image data efficiently
  writeImageData(imageData: ImageData): number {
    const size = imageData.data.length;
    const ptr = this.exports.alloc(size);
    
    const wasmBuffer = new Uint8ClampedArray(this.memory.buffer, ptr, size);
    wasmBuffer.set(imageData.data);
    
    return ptr;
  }
  
  // Read processed image back
  readImageData(ptr: number, size: number): Uint8ClampedArray {
    return new Uint8ClampedArray(this.memory.buffer, ptr, size);
  }
}
```

## 53.2 Rust Integration

Setting up the Rust toolchain for WebAssembly.

### Project Setup with wasm-pack

Creating a Rust library for web consumption:

```toml
# Cargo.toml
[package]
name = "image-processor"
version = "0.1.0"
edition = "2021"
crate-type = ["cdylib"]

[dependencies]
wasm-bindgen = "0.2"
js-sys = "0.3"
web-sys = "0.3"

# Image processing crate (Rust native)
image = { version = "0.24", default-features = false, features = ["png", "jpeg"] }

[dependencies.web-sys]
version = "0.3"
features = [
  "console",
  "CanvasRenderingContext2d",
  "ImageData",
  "HtmlCanvasElement",
]

[profile.release]
# Optimize for size (critical for web)
opt-level = "z"
lto = true
codegen-units = 1
panic = "abort"
```

```rust
// src/lib.rs
use wasm_bindgen::prelude::*;
use image::{ImageFormat, DynamicImage, imageops::FilterType};

// When the WASM module is loaded, run this
#[wasm_bindgen(start)]
pub fn start() {
    console_log::init_with_level(log::Level::Debug).ok();
}

#[wasm_bindgen]
pub struct ImageResizer {
    image: Option<DynamicImage>,
}

#[wasm_bindgen]
impl ImageResizer {
    #[wasm_bindgen(constructor)]
    pub fn new() -> ImageResizer {
        ImageResizer { image: None }
    }
    
    // Load image from bytes
    pub fn load_from_bytes(&mut self, bytes: &[u8], format: &str) -> Result<(), JsValue> {
        let format = match format {
            "png" => ImageFormat::Png,
            "jpg" | "jpeg" => ImageFormat::Jpeg,
            _ => return Err(JsValue::from_str("Unsupported format")),
        };
        
        let img = image::load_from_memory_with_format(bytes, format)
            .map_err(|e| JsValue::from_str(&format!("{:?}", e)))?;
        
        self.image = Some(img);
        Ok(())
    }
    
    // Resize maintaining aspect ratio
    pub fn resize(&mut self, max_width: u32, max_height: u32) -> Result<(), JsValue> {
        let img = self.image.as_ref()
            .ok_or(JsValue::from_str("No image loaded"))?;
        
        let (width, height) = (img.width(), img.height());
        
        // Calculate new dimensions
        let ratio = (max_width as f32 / width as f32)
            .min(max_height as f32 / height as f32);
        
        let new_width = (width as f32 * ratio) as u32;
        let new_height = (height as f32 * ratio) as u32;
        
        let resized = img.resize(new_width, new_height, FilterType::Lanczos3);
        self.image = Some(resized);
        Ok(())
    }
    
    // Export as JPEG bytes
    pub fn to_jpeg_bytes(&self, quality: u8) -> Result<Vec<u8>, JsValue> {
        let img = self.image.as_ref()
            .ok_or(JsValue::from_str("No image loaded"))?;
        
        let mut buffer = Vec::new();
        let encoder = image::codecs::jpeg::JpegEncoder::new_with_quality(
            &mut buffer, 
            quality
        );
        
        img.write_with_encoder(encoder)
            .map_err(|e| JsValue::from_str(&format!("{:?}", e)))?;
        
        Ok(buffer)
    }
    
    // Get dimensions
    pub fn width(&self) -> Option<u32> {
        self.image.as_ref().map(|i| i.width())
    }
    
    pub fn height(&self) -> Option<u32> {
        self.image.as_ref().map(|i| i.height())
    }
}
```

### Build Configuration

Compiling for the web target:

```json
// package.json build scripts
{
  "scripts": {
    "wasm:build": "wasm-pack build --target web --out-dir pkg",
    "wasm:build:release": "wasm-pack build --target web --release --out-dir pkg",
    "dev": "npm run wasm:build && next dev",
    "build": "npm run wasm:build:release && next build"
  }
}
```

```javascript
// next.config.js
const path = require('path');

module.exports = {
  webpack: (config, { isServer }) => {
    // WASM support
    config.experiments = {
      ...config.experiments,
      asyncWebAssembly: true,
      layers: true,
    };
    
    // Handle .wasm files
    config.module.rules.push({
      test: /\.wasm$/,
      type: 'webassembly/async',
    });
    
    // Alias for Rust pkg
    config.resolve.alias['@wasm'] = path.join(__dirname, 'rust/pkg');
    
    return config;
  },
};
```

## 53.3 Performance-Critical Modules

Implementing compute-intensive features in WASM.

### High-Performance Hashing

Cryptographic operations in Rust:

```rust
// src/hash.rs
use wasm_bindgen::prelude::*;
use sha2::{Sha256, Digest};
use blake3;

#[wasm_bindgen]
pub fn sha256_hash(data: &[u8]) -> Vec<u8> {
    let mut hasher = Sha256::new();
    hasher.update(data);
    hasher.finalize().to_vec()
}

#[wasm_bindgen]
pub fn blake3_hash(data: &[u8]) -> String {
    let hash = blake3::hash(data);
    hash.to_hex().to_string()
}

// Parallel hashing using Rayon (requires WASM threads)
#[wasm_bindgen]
pub fn batch_sha256(data: Vec<js_sys::Uint8Array>) -> Vec<js_sys::Uint8Array> {
    data.into_iter()
        .map(|arr| {
            let slice = arr.to_vec();
            let result = sha256_hash(&slice);
            js_sys::Uint8Array::from(&result[..])
        })
        .collect()
}
```

```typescript
// lib/crypto/wasm-crypto.ts
import dynamic from 'next/dynamic';

// Dynamic import to avoid SSR issues
export async function loadCryptoWasm() {
  const wasm = await import('@wasm');
  await wasm.default(); // Initialize WASM module
  
  return {
    sha256: (data: Uint8Array) => {
      const result = wasm.sha256_hash(data);
      return new Uint8Array(result);
    },
    
    blake3: (data: Uint8Array): string => {
      return wasm.blake3_hash(data);
    },
    
    // Benchmark comparison
    benchmark: (data: Uint8Array, iterations: number = 1000) => {
      const startWasm = performance.now();
      for (let i = 0; i < iterations; i++) {
        wasm.sha256_hash(data);
      }
      const wasmTime = performance.now() - startWasm;
      
      // Compare with Web Crypto API
      const startWeb = performance.now();
      for (let i = 0; i < iterations; i++) {
        crypto.subtle.digest('SHA-256', data);
      }
      const webTime = performance.now() - startWeb;
      
      return { wasmTime, webTime, speedup: webTime / wasmTime };
    },
  };
}

// React hook for WASM crypto
'use client';

import { useEffect, useState, useCallback } from 'react';

export function useWasmCrypto() {
  const [crypto, setCrypto] = useState<any>(null);
  const [loading, setLoading] = useState(true);
  
  useEffect(() => {
    loadCryptoWasm().then((wasm) => {
      setCrypto(wasm);
      setLoading(false);
    });
  }, []);
  
  const hashFile = useCallback(async (file: File): Promise<string> => {
    if (!crypto) throw new Error('WASM not loaded');
    
    const arrayBuffer = await file.arrayBuffer();
    const uint8Array = new Uint8Array(arrayBuffer);
    
    return crypto.blake3(uint8Array);
  }, [crypto]);
  
  return { hashFile, loading, benchmark: crypto?.benchmark };
}
```

### Image Processing Pipeline

Client-side image optimization:

```typescript
// components/image-processor.tsx
'use client';

import { useState, useCallback, useRef } from 'react';
import { loadImageProcessor } from '@/lib/wasm/image-processor';

interface ProcessedImage {
  originalSize: number;
  processedSize: number;
  width: number;
  height: number;
  blob: Blob;
  url: string;
}

export function ImageProcessor() {
  const [processing, setProcessing] = useState(false);
  const [result, setResult] = useState<ProcessedImage | null>(null);
  const [error, setError] = useState<string | null>(null);
  const processorRef = useRef<any>(null);
  
  // Initialize WASM processor
  useState(() => {
    loadImageProcessor().then(p => {
      processorRef.current = p;
    });
  });
  
  const processImage = useCallback(async (file: File) => {
    if (!processorRef.current) {
      setError('Processor not ready');
      return;
    }
    
    setProcessing(true);
    setError(null);
    
    try {
      const arrayBuffer = await file.arrayBuffer();
      const uint8Array = new Uint8Array(arrayBuffer);
      
      // Process in WASM
      const processor = processorRef.current;
      await processor.load_from_bytes(uint8Array, file.type.split('/')[1]);
      
      // Resize if too large
      const originalWidth = processor.width();
      const originalHeight = processor.height();
      
      if (originalWidth > 1920 || originalHeight > 1080) {
        processor.resize(1920, 1080);
      }
      
      // Export as optimized JPEG
      const jpegBytes = processor.to_jpeg_bytes(85);
      const blob = new Blob([jpegBytes], { type: 'image/jpeg' });
      
      setResult({
        originalSize: file.size,
        processedSize: blob.size,
        width: processor.width(),
        height: processor.height(),
        blob,
        url: URL.createObjectURL(blob),
      });
    } catch (err) {
      setError(err instanceof Error ? err.message : 'Processing failed');
    } finally {
      setProcessing(false);
    }
  }, []);
  
  const handleDrop = useCallback((e: React.DragEvent) => {
    e.preventDefault();
    const file = e.dataTransfer.files[0];
    if (file && file.type.startsWith('image/')) {
      processImage(file);
    }
  }, [processImage]);
  
  return (
    <div 
      onDrop={handleDrop}
      onDragOver={(e) => e.preventDefault()}
      className="border-2 border-dashed p-8 text-center"
    >
      <input
        type="file"
        accept="image/*"
        onChange={(e) => e.target.files?.[0] && processImage(e.target.files[0])}
        className="hidden"
        id="image-input"
      />
      <label htmlFor="image-input" className="cursor-pointer">
        {processing ? 'Processing...' : 'Drop image here or click to upload'}
      </label>
      
      {result && (
        <div className="mt-4">
          <img src={result.url} alt="Processed" className="max-w-md mx-auto" />
          <p className="text-sm text-gray-600 mt-2">
            {Math.round(result.originalSize / 1024)}KB → {Math.round(result.processedSize / 1024)}KB
            ({Math.round((1 - result.processedSize / result.originalSize) * 100)}% reduction)
          </p>
        </div>
      )}
    </div>
  );
}
```

## 53.4 WASM with Next.js

Integration patterns for the Next.js ecosystem.

### Dynamic Loading Strategy

Avoiding SSR pitfalls with client-side only loading:

```typescript
// lib/wasm/loader.ts
'use client';

import type { InitOutput } from '@wasm';

let wasmModule: InitOutput | null = null;
let initializationPromise: Promise<InitOutput> | null = null;

export async function initWasm(): Promise<InitOutput> {
  if (wasmModule) return wasmModule;
  if (initializationPromise) return initializationPromise;
  
  // Dynamic import ensures this only runs on client
  initializationPromise = import('@wasm').then(async (module) => {
    await module.default();
    wasmModule = module;
    return module;
  });
  
  return initializationPromise;
}

// Hook for components
export function useWasm() {
  const [ready, setReady] = useState(false);
  const [error, setError] = useState<Error | null>(null);
  
  useEffect(() => {
    initWasm()
      .then(() => setReady(true))
      .catch(setError);
  }, []);
  
  return { ready, error, module: wasmModule };
}

// Route segment config for WASM routes
// app/tools/image-optimizer/page.tsx
export const dynamic = 'force-dynamic';
export const runtime = 'nodejs'; // WASM runs in browser, not server
```

### Streaming Compilation

Progressive enhancement for large WASM modules:

```typescript
// components/wasm-loader.tsx
'use client';

import { useEffect, useState } from 'react';
import { Progress } from '@/components/ui/progress';

interface WasmLoaderProps {
  children: (wasm: any) => React.ReactNode;
  moduleName: string;
  sizeMB?: number;
}

export function WasmLoader({ children, moduleName, sizeMB = 2 }: WasmLoaderProps) {
  const [progress, setProgress] = useState(0);
  const [wasm, setWasm] = useState<any>(null);
  const [error, setError] = useState<string | null>(null);
  
  useEffect(() => {
    const load = async () => {
      try {
        // Fetch with progress tracking
        const response = await fetch(`/wasm/${moduleName}.wasm`);
        const contentLength = +(response.headers.get('content-length') || 0);
        
        if (!response.body) throw new Error('No response body');
        
        const reader = response.body.getReader();
        let received = 0;
        const chunks: Uint8Array[] = [];
        
        while (true) {
          const { done, value } = await reader.read();
          if (done) break;
          
          chunks.push(value);
          received += value.length;
          
          if (contentLength) {
            setProgress(Math.round((received / contentLength) * 100));
          }
        }
        
        // Concatenate chunks
        const allChunks = new Uint8Array(received);
        let position = 0;
        for (const chunk of chunks) {
          allChunks.set(chunk, position);
          position += chunk.length;
        }
        
        // Compile and instantiate
        const module = await WebAssembly.compile(allChunks);
        const instance = await WebAssembly.instantiate(module, {
          env: {
            memory: new WebAssembly.Memory({ initial: 256, maximum: 512 }),
          },
        });
        
        setWasm(instance.exports);
      } catch (err) {
        setError(err instanceof Error ? err.message : 'Failed to load WASM');
      }
    };
    
    load();
  }, [moduleName]);
  
  if (error) return <div className="text-red-500">Error: {error}</div>;
  if (!wasm) return (
    <div className="p-4">
      <p className="mb-2">Loading WASM module ({sizeMB}MB)...</p>
      <Progress value={progress} />
    </div>
  );
  
  return <>{children(wasm)}</>;
}
```

## 53.5 Use Cases

Real-world scenarios for WASM in production.

### Data Compression

Client-side zip/gzip operations:

```rust
// Rust implementation using flate2
use wasm_bindgen::prelude::*;
use flate2::write::{GzEncoder, ZlibEncoder};
use flate2::read::{GzDecoder, ZlibDecoder};
use flate2::Compression;
use std::io::{Write, Read};

#[wasm_bindgen]
pub fn compress_gzip(data: &[u8], level: u32) -> Vec<u8> {
    let compression = match level {
        0 => Compression::none(),
        1 => Compression::fast(),
        9 => Compression::best(),
        _ => Compression::new(level),
    };
    
    let mut encoder = GzEncoder::new(Vec::new(), compression);
    encoder.write_all(data).unwrap();
    encoder.finish().unwrap()
}

#[wasm_bindgen]
pub fn decompress_gzip(data: &[u8]) -> Result<Vec<u8>, JsValue> {
    let mut decoder = GzDecoder::new(data);
    let mut result = Vec::new();
    decoder.read_to_end(&mut result)
        .map_err(|e| JsValue::from_str(&format!("Decompression failed: {}", e)))?;
    Ok(result)
}
```

```typescript
// Usage for offline storage compression
export async function compressForStorage(data: object): Promise<Uint8Array> {
  const wasm = await import('@wasm/compression');
  await wasm.default();
  
  const jsonString = JSON.stringify(data);
  const encoder = new TextEncoder();
  const bytes = encoder.encode(jsonString);
  
  // Compress to ~10-20% of original size
  const compressed = wasm.compress_gzip(bytes, 6);
  
  return compressed;
}
```

### Game Physics

Running physics simulations:

```typescript
// Physics engine wrapper for canvas games
'use client';

import { useRef, useEffect } from 'react';

interface PhysicsBody {
  x: number;
  y: number;
  vx: number;
  vy: number;
  mass: number;
}

export function useWasmPhysics() {
  const worldRef = useRef<any>(null);
  const bodiesRef = useRef<Map<string, number>>(new Map());
  
  useEffect(() => {
    import('@wasm/physics').then((module) => {
      module.default();
      worldRef.current = module.PhysicsWorld.new(0.0, -9.8); // gravity
    });
    
    return () => {
      // Cleanup
      worldRef.current?.free();
    };
  }, []);
  
  const addBody = (id: string, body: PhysicsBody) => {
    if (!worldRef.current) return;
    
    const handle = worldRef.current.add_body(
      body.x, body.y, 
      body.vx, body.vy,
      body.mass
    );
    bodiesRef.current.set(id, handle);
  };
  
  const step = (dt: number) => {
    if (!worldRef.current) return [];
    
    worldRef.current.step(dt);
    
    // Get updated positions
    const positions: Array<{ id: string; x: number; y: number }> = [];
    bodiesRef.current.forEach((handle, id) => {
      const pos = worldRef.current.get_body_position(handle);
      positions.push({ id, x: pos.x, y: pos.y });
    });
    
    return positions;
  };
  
  return { addBody, step };
}
```

## 53.6 Debugging WASM

Development and production debugging strategies.

### Source Maps and DevTools

Setting up debug symbols:

```toml
# Cargo.toml
[package.metadata.wasm-pack.profile.debugging]
wasm-opt = ["-g", "--disable-threads"]

[profile.dev]
debug = true
opt-level = 0

[profile.release]
# Keep debug symbols for production debugging
debug = 1
```

```typescript
// Enable detailed stack traces
import init from '@wasm';

export async function initWithDebug() {
  // In development, load source maps
  if (process.env.NODE_ENV === 'development') {
    // Chrome DevTools will map WASM addresses to Rust source
    console.log('WASM source maps enabled');
  }
  
  await init();
}

// Error handling with context
try {
  wasm.risky_operation();
} catch (e) {
  // Rust panics become JavaScript exceptions
  console.error('WASM Panic:', e);
  // Stack trace points to Rust source file and line number
}
```

### Performance Profiling

Benchmarking WASM vs JavaScript:

```typescript
// lib/benchmark.ts
interface BenchmarkResult {
  name: string;
  jsTime: number;
  wasmTime: number;
  speedup: number;
  iterations: number;
}

export function benchmark(
  name: string,
  jsFn: () => void,
  wasmFn: () => void,
  iterations: number = 1000
): BenchmarkResult {
  // Warmup
  for (let i = 0; i < 100; i++) {
    jsFn();
    wasmFn();
  }
  
  // Benchmark JS
  const jsStart = performance.now();
  for (let i = 0; i < iterations; i++) {
    jsFn();
  }
  const jsTime = performance.now() - jsStart;
  
  // Benchmark WASM
  const wasmStart = performance.now();
  for (let i = 0; i < iterations; i++) {
    wasmFn();
  }
  const wasmTime = performance.now() - wasmStart;
  
  return {
    name,
    jsTime,
    wasmTime,
    speedup: jsTime / wasmTime,
    iterations,
  };
}

// Component for visualizing results
export function BenchmarkResults({ results }: { results: BenchmarkResult[] }) {
  return (
    <div className="space-y-4">
      {results.map((r) => (
        <div key={r.name} className="border p-4 rounded">
          <h3 className="font-bold">{r.name}</h3>
          <div className="grid grid-cols-3 gap-4 mt-2">
            <div>
              <div className="text-sm text-gray-500">JavaScript</div>
              <div className="text-lg">{r.jsTime.toFixed(2)}ms</div>
            </div>
            <div>
              <div className="text-sm text-gray-500">WebAssembly</div>
              <div className="text-lg">{r.wasmTime.toFixed(2)}ms</div>
            </div>
            <div>
              <div className="text-sm text-gray-500">Speedup</div>
              <div className="text-lg font-bold text-green-600">
                {r.speedup.toFixed(2)}x
              </div>
            </div>
          </div>
        </div>
      ))}
    </div>
  );
}
```

## 53.7 Future of WASM

Emerging standards and capabilities.

### WASI (WebAssembly System Interface)

Server-side WASM potential:

```typescript
// Future: WASI for Next.js API routes
// app/api/compute/route.ts (hypothetical future)
import { WASI } from 'wasi';

export const runtime = 'wasi'; // Future Next.js runtime option

export async function POST(request: Request) {
  // Run WASM with filesystem and network access
  const wasi = new WASI({
    args: [],
    env: {},
    preopens: {
      '/sandbox': '/tmp',
    },
  });
  
  const wasm = await WebAssembly.compile(
    await fetch('/wasm/sandboxed-tool.wasm').then(r => r.arrayBuffer())
  );
  
  const instance = await WebAssembly.instantiate(wasm, {
    wasi_snapshot_preview1: wasi.wasiImport,
  });
  
  wasi.start(instance);
  
  return Response.json({ result: 'computed securely' });
}
```

### Component Model

Composability and language interop:

```typescript
// Future: WebAssembly Components
// Interface definitions (WIT)
/*
package example:calculator;

world calculator {
  import print: func(msg: string);
  
  export add: func(a: s32, b: s32) -> s32;
  export multiply: func(a: s32, b: s32) -> s32;
}
*/

// JavaScript consumption of components
import { createCalculator } from '@wasm/calculator';

const calc = await createCalculator({
  print: (msg: string) => console.log(msg),
});

const result = calc.add(2, 3); // Type-safe calls across language boundary
```

## Key Takeaways from Chapter 53

1. **WebAssembly Architecture**: WASM provides a sandboxed, stack-based virtual machine with linear memory (contiguous array buffer) shared between JavaScript and native code. Understand the boundary crossing costs—while computation is fast, frequent JS/WASM calls for small operations can negate performance benefits.

2. **Rust Integration**: Use `wasm-bindgen` to generate JavaScript bindings automatically from Rust code. Configure `Cargo.toml` with `opt-level = "z"` and `lto = true` for minimal bundle sizes (critical for web delivery), and use `wasm-pack` to build npm-compatible packages.

3. **Next.js Integration**: Always load WASM modules dynamically using `import()` in client components to avoid SSR errors (WASM requires Web APIs). Use `WebAssembly.compile()` with streaming fetch for progressive loading of large modules, showing progress indicators for multi-megabyte binaries.

4. **Memory Management**: Pass data efficiently by writing directly to WASM linear memory using `Uint8Array` views rather than copying. For large binary data (images, files), pass pointers to Rust and operate in-place to avoid double memory usage during processing.

5. **Performance Optimization**: WASM excels at compute-intensive tasks: cryptographic hashing (2-10x faster than JS), image resizing (using Rust's `image` crate), data compression (gzip/zlib), and mathematical simulations. Avoid using WASM for simple DOM manipulation or network requests where JavaScript V8 optimizations perform better.

6. **Debugging Setup**: Enable debug symbols in `Cargo.toml` (`debug = 1`) and use Chrome DevTools' WASM debugging to set breakpoints in Rust source code. Implement comprehensive error handling on the Rust side using `Result<T, JsValue>` to translate panics into catchable JavaScript exceptions.

7. **Future Standards**: Monitor WASI (WebAssembly System Interface) for server-side WASM capabilities allowing sandboxed execution with filesystem access, and the Component Model proposal for type-safe language interoperability. These may enable edge-computed WASM in Next.js API routes in future platform versions.

## Coming Up Next

**Chapter 54: Future of Next.js**

As we conclude this comprehensive guide, Chapter 54 looks ahead to the evolving landscape of React and Next.js. We'll explore upcoming features in React 19 and Next.js 15+, the evolution of Server Components architecture, emerging rendering patterns, and how AI integration is reshaping web development. Learn to future-proof your applications and stay ahead of the framework's rapid innovation cycle.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='52. advanced_performance_techniques.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='54. future_of_nextjs.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
